<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=300 height=300 align=left class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Spring 2023<br>
<font color=3C99D size=5>
    Practical Assignment 3 - Reinforcement Learning <br>
<font color=696880 size=4>
    Mohammad Moshtaghi - Ali Salesi - Hossein Goli

____

# Personal Data

In [ ]:
student_number = '400104715'
first_name = 'Ali'
last_name = 'Aghayari'

# Rules
- Make sure that all of your cells can be run perfectly. 

# Q2: Sentence Generator (100 Points)

<font size=4>
Author: Ali Salesi
<br/>
<font color=red>
Please run all the cells.
</font>
</font>
<br/>
</div>

In this assignment we implement a text generator using RL.

## Preprocess

### Dataset

First, lets download the text corpus crawled from `VOA Persian` from 2003 to 2008.

In [ ]:
!wget -O "voa_persian.txt" "https://storage.googleapis.com/danielk-files/farsi-text/merged_files/voa_persian_2003_2008_cleaned.txt"

--2023-05-14 16:29:11--  https://storage.googleapis.com/danielk-files/farsi-text/merged_files/voa_persian_2003_2008_cleaned.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 142.250.141.128, 2607:f8b0:4023:c0d::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69708061 (66M) [text/plain]
Saving to: ‘voa_persian.txt’

voa_persian.txt     100%[===================>]  66.48M   161MB/s    in 0.4s    

2023-05-14 16:29:12 (161 MB/s) - ‘voa_persian.txt’ saved [69708061/69708061]



In [ ]:
!wc -l voa_persian.txt | awk '{print $1}'

488253


In [ ]:
!head voa_persian.txt

پيمان صلح بين ژاپن و روسيه
بنا به گزارشهای منتشره در توکيو، ژاپن و روسيه در زمينه يک پيمان صلح در چارچوبی گسترده توافق کرده اند که رسماً به مخاصمات جنگ دوم جهانی ميان دو کشور پايان خواهند داد.

در يکی از اين گزارشها، که از سوی خبرگزاری کيودُو،انتشار يافته، گفته شده است که دو کشور برای رفع اختلافات ديرين خود بر سر چهار جزيره از جزاير زنجيره ای کوريل، بر اساس سه پيمان گذشته خود عمل خواهند کرد.
بموجب يکی از اين پيمانها که در سال ۱۹۵۶ امضاء شده، دو تا از اين جزيره ها پس از امضاء يک پيمان صلح به ژاپن پس داده خواهد شد.
اما بموجب پيمانی که در سال ۱۹۹۳ به امضاء رسيده، مسئله حاکميت اين چهار جزيره بايستی پيش از امضاء پيمان صلح فيصله يابد.
هيچ يک از دو طرف نحوه استفاده از پيمان های پيشين را اعلام نکرده اند.

تشکيلات فلسطينی نخستين بودجه رسمی خود را اعلام کرد
تشکيلات فلسطينی پس از دو سال نخستين بودجه رسمی خود را اعلام کرد و قول داد برای از ميان برداشتن فساد و پاسخگوئی بيشتر به مردم تلاش کند.


### Normalization

Then we have to normalize and lemmatize the text so we can have a better generalization of semantics in prompt generation.

We'll use `hazm` library for this purpose.

In [ ]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from __future__ import unicode_literals
from hazm import Normalizer, Lemmatizer, word_tokenize
from tqdm import tqdm
import re

normalizer = Normalizer()
lemmatizer = Lemmatizer()


def normalize(line: str):
    line = re.sub(
        r'[.{}[\]؛:«»؟!٬٫٪×،*)(ـ+<>\'",`=+\-?!@#$%^&*()_\/\\\\]', '', line.strip())
    line = re.sub(r'\s+', ' ', line.strip())
    line = normalizer.normalize(line)
    words = word_tokenize(line)
    words = [lemmatizer.lemmatize(word) for word in words]
    line = ' '.join(words)
    return line


In [ ]:
normalize('من خیلی خوشحال هستم و کتاب‌های زیادی درباره یخچال‌های قطبی خوانده‌ام.')

'من خیلی خوشحال #هست و کتاب زیاد درباره یخچال قطب خواند#خوان'

In [ ]:
voa = open('voa_persian.txt')
voa_norm = open('voa_persian_normalized.txt', 'w')
for i, line in tqdm(enumerate(voa), total=488253):
    voa_norm.write(normalize(line) + '\n')


100%|██████████| 488253/488253 [01:02<00:00, 7836.79it/s]


In [ ]:
!head voa_persian_normalized.txt

پیمان صلح بین ژاپن و روسیه
بنا به گزارش منتشره در توکیو ژاپن و روسیه در زمینه یک پیمان صلح در چارچوب گسترده توافق کرد#کن که رسما به مخاصمات جنگ دوم جهانی میان دو کشور پایان داد#ده

در یک از این گزارش که از سو خبرگزاری کیودوانتشار یافته گفت#گو که دو کشور برای رفع اختلافات دیرین خود بر سر چهار جزیره از جزایر زنجیره کوریل بر اساس سه پیمان گذشته خود عمل کرد#کن
بموجب یک از این پیمان که در سال ۱۹۵۶ امضاء شده دو تا از این جزیره پس از امضاء یک پیمان صلح به ژاپن پس داد#ده
اما بموجب پیمان که در سال ۱۹۹۳ به امضاء رسیده مسئله حاکمیت این چهار جزیره ایستاد#ایست پیش از امضاء پیمان صلح فیصله یافت#یاب
هیچ یک از دو طرف نحوه استفاده از پیمان پیشین را اعلام کرد#کن

تشکیلات فلسطین نخستین بودجه رسم خود را اعلام کرد#کن
تشکیلات فلسطین پس از دو سال نخستین بودجه رسم خود را اعلام کرد#کن و قول داد برای از میان برداشتن فساد و پاسخگوئی بیشتر به مردم تلاش کند


### Language Model

Now we'll use `KenLM` to train an N-gram language model. an N-gram model calculates probability of N words being together.

You can read more about N-gram [here](https://towardsdatascience.com/understanding-word-n-grams-and-n-gram-probability-in-natural-language-processing-9d9eef0fa058).

First, let's install download and build `KenLM`.

In [ ]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz; mkdir kenlm/build; cd kenlm/build; cmake ..; make -j2

--2023-05-14 16:54:01--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491888 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 480.36K  1.62MB/s    in 0.3s    

2023-05-14 16:54:01 (1.62 MB/s) - written to stdout [491888/491888]

mkdir: cannot create directory ‘kenlm/build’: File exists
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Configuring done
-- Generating done
-- Build files have been written to: /content/kenlm/build
[ 38%] Built target kenlm_util
[ 41%] Built target probing_hash_table_benchmark
[ 46%] Built target kenlm_filter
[ 71%] Built target kenlm
[ 76%] Built target fragment
[ 76%] Built target query
[ 78%] Built target build_binary
[ 81%] Built target kenlm_benchmark
[ 83%] Built target filter
[ 92%] Built target kenlm_builder
[ 95%] Built tar

Now let's make a 5-gram model using 

In [ ]:
!kenlm/build/bin/lmplz -o 5 <"voa_persian_normalized.txt"> "voa_persian.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /content/voa_persian_normalized.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 7151925 types 105480
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:1265760 2:1062379968 3:1991962624 4:3187139840 5:4647912448
Statistics:
1 105480 D1=0.692744 D2=1.02088 D3+=1.36881
2 1273882 D1=0.753645 D2=1.09856 D3+=1.34093
3 3443025 D1=0.837135 D2=1.17753 D3+=1.39364
4 5019416 D1=0.905518 D2=1.28914 D3+=1.4375
5 5611302 D1=0.891822 D2=1.51475 D3+=1.61147
Memory estimate for binary LM:
type     MB
probing 321 assuming -p 1.5
probing 377 assuming -r models -p 1.5
trie    153 without quantization
trie     83 assuming -q 8 -b 8 quantization 
trie    135 assuming -a 22 array pointer compression
trie     66 assuming -a 22 -q 8 -b 8 array pointer compression and quantiza

In [ ]:
!head -n 20 voa_persian.arpa

\data\
ngram 1=105480
ngram 2=1273882
ngram 3=3443025
ngram 4=5019416
ngram 5=5611302

\1-grams:
-6.13853	<unk>	0
0	<s>	-1.5815215
-2.112993	</s>	0
-3.5871985	پیمان	-0.5081966
-3.3047166	صلح	-0.5604726
-2.8914635	بین	-0.67792463
-3.3033433	ژاپن	-0.5074516
-2.0037067	و	-0.8103022
-3.0975704	روسیه	-0.56394786
-3.6942556	بنا	-0.50759333
-2.0797186	به	-1.0189861
-3.0476315	گزارش	-0.6365013


Now lets extract the list of words and sort them using their probabilities.

In [ ]:
words = []
words_started = False
with open('voa_persian.arpa') as f:
    for line in f:
        line = line.strip()
        if not words_started:
            if line == r'\1-grams:':
                words_started = True
        else:
            if line == r'\2-grams:':
                words = words[:-1]
                break
            words.append(line.split())
words_sorted = sorted(words, key=lambda x: x[0])
words_total = [w[1] for w in words_sorted]
words_total.remove('</s>')
words_total.insert(0, '</s>')
words_total[:20]


['</s>',
 'در',
 'و',
 'به',
 'را',
 'که',
 'از',
 'با',
 '#است',
 'بود#باش',
 'یک',
 'برای',
 'این',
 'شد#شو',
 'گفت#گو',
 'خود',
 'آن',
 'کرد#کن',
 'روز',
 'نیز']

In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/kpu/kenlm/archive/master.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import kenlm

model = kenlm.Model('voa_persian.arpa')

Now we need a measure using our language model to measure how well our sentence fit together. Our model can measure the probability of a sentence using N-gram.

This has a downside. the longer the sentence gets, the lower its' probability becomes. We don't want that. So we introduce `perplexity`. a measure which is normalized by the sentence's length. Lower perplexity means the semantics of our sentence fits better together.

You can read more about perplexity [here](https://medium.com/nlplanet/two-minutes-nlp-perplexity-explained-with-simple-probabilities-6cdc46884584).
$$
\begin{align}
PP(S) &= 10 ^ {-\frac{log(P(S))}{N}} \\
PP(S) &= \sqrt[N]{\frac{1}{P(S)}} \\
PP(S) &= \sqrt[N]{\frac{1}{P(W_1W_2...W_N)}} \\
PP(S) &= \sqrt[N]{\prod_{i=1}^N{\frac{1}{P(W_i|W_1W_2...W_{i-1})}}}
\end{align}
$$
**Note**: `KenLM` score function return log10 probability of a sentence.

### Perplexity (10 Points)

In [ ]:
def perplexity(sentence: str):
    """
    returns the perplexity of a sentence using model.score method
    Args:
      sentence: string of words

    Returns:
      perplexity: 10^(-lop10p(sentence) / N)
    """
    # help from : https://stackoverflow.com/questions/43841467/how-to-compute-perplexity-using-kenlm
    words = len(sentence.split()) + 1 # For </s>
    return 10.0**(- model.score(sentence) / words)


In [ ]:
sen_1 = normalize('من خوشحال شدم')
sen_2 = normalize('من خودکار شدم')
sen_3 = normalize('من کتاب یخچال')
sen_4 = normalize('نستب سنبتس سنمبتم')


In [ ]:
print(sen_1, perplexity(sen_1))
print(sen_2, perplexity(sen_2))
print(sen_3, perplexity(sen_3))
print(sen_4, perplexity(sen_4))


من خوشحال شد#شو 137.01680134111533
من خودکار شد#شو 1222.8542216279307
من کتاب یخچال 7467.088563904955
نستب سنبتس سنمبتم 336919.6792991999


## Reinforcement Learning

### Reward Function (10 Points)

Reward function should give us a reward based on how the last word added to the sentence changed the meaning and how well it fits with the others.

In [ ]:
def reward(base_sentence: str, new_word: str):
    """
    returns the reward of adding a new word to a base sentence
    Args:
      base_sentence: string of words up until now
      new_word: new word to be added to the base sentence

    Returns:
      reward: change of perplexity of the base sentence after adding the new word. positive reward means the new word is more likely to be added to the base sentence.
    """
    # need it ti be closer to 1 so we will give it a - property for now
    if new_word == '' or new_word == ' ' : return 0
    if base_sentence == '' or base_sentence == ' ' : -(perplexity(new_word) - perplexity(base_sentence))
    return -(perplexity(base_sentence + " " + new_word) - perplexity(base_sentence))


In [ ]:
print(reward('', 'من'))
print(reward('من', 'خوشحال'))
print(reward('من خوشحال', 'شد#شو'))
print(reward('جنگ جهانی', 'اول'))
print(reward('جنگ جهانی', 'دوم'))
print(reward('جنگ جهانی', 'صورتی'))


-690.0787915528637
-167.71077564865902
723.9550949735619
457.5602143706758
483.02565363284526
-4981.013565729618


Since we have to implement text generator using a tabular implementation, we have to assume that all that matters in a text is in a window of N words. It matches our language model of N-gram.

We model it using MDP. the first state is `<s>` state. it has no text and 0 perplexity. The next state is $W_1$ state. We usually have a negative perplexity because no text has more meaning than a one word sentence. Next is $W_1W_2$ state until we reach $W_1W_2...W_N$ state, from then with our window assumption we go to $W_2W_3...W_{N+1}$ state and $W_3W_4...W_{N+2}$ and so on.

First thing we notice is that our search space is **really** big. Each word choice has thousands of possibilites. We cannot model our search space using our normal Q Table.
Since our states are sequential and we need to find the best word using our current state, we can use `dict` in `dict` architecture.

First we reduce the search space to the 10K most used words.
For faster computation, we use each word index for states.

### Utility Functions (10 Points)

In [ ]:
words = words_total[:10000]
# 0 index is for </s> which means end of the sentence.
indexes = dict()
for i, w in enumerate(words):
    indexes[w] = i


def index_to_word(index: int):
    """
    returns the word of a given index
    Args:
        index: index of the word

    Returns:
        word: word of the given index. '.' if the index is 0 (end of sentence or </s>)
    """
    if index is None : return ""
    if index > len(words_total) : raise Exception("index out of bound")
    if index == 0 : return '.'
    return words[index]


def word_to_index(word: str):
    """
    returns the index of a given word
    Args:
        word: word of the given index. word should be normalized.

    Returns:
        index: index of the word. -1 if the word is not in the vocabulary
    """
    normalize(word)
    if word not in indexes : return -1
    if word == '.' : return 0
    return indexes[word]


def state_to_sentence(state: list[int]):
    """
    returns the sentence of a given state
    Args:
        state: list of indexes of words

    Returns:
        sentence: string of words. '.' when the state is 0 (end of sentence or </s>)
    """
    return " ".join([index_to_word(x) for x in state])

def sentence_to_state(sentence: str):
    """
    returns the state of a given sentence
    Args:
        sentence: string of words. sentence should be normalized.

    Returns:
        state: list of indexes of words. no need to add the index of </s> (end of sentence) to the state
    """
    sentence = normalize(sentence)
    if sentence[-1] == '\s' or  sentence[-1] == '.' :  sentence = sentence[:-1]
    return [word_to_index(x) for x in sentence.split()]


print(index_to_word(10))
print(word_to_index('یک'))
print(state_to_sentence([390, 2884, 24, 0]))
print(sentence_to_state('من خوشحال هستم'))
print(state_to_sentence([390, 10, 791, 3816]))
print(sentence_to_state('من یک کتاب خریدم'))

یک
10
مقام کارت رئیس .
[389, 2887, 23]
مقام یک برقرار محترم
[389, 10, 787, 3809]


In [ ]:
# example Q Table
q_table = {
    word_to_index('من'): (10, {
        word_to_index('خوشحال'): (20, {
            word_to_index('هستم'): (25, {
                0: (0, {}),
            }),
        }),
        word_to_index('یک'): (5, {
            word_to_index('کتاب'): (15, {
                word_to_index('خریدم'): (10, {}),
            }),
            word_to_index('گل'): (15, {
                word_to_index('دیدم'): (8, {}),
            }),
        })
    }),
    word_to_index('تو'): (10, {
        word_to_index('خوشحال'): (20, {
            word_to_index('هستی'): (7, {
                0: (0, {}),
            }),
        }),
        word_to_index('دو'): (5, {
            word_to_index('کتاب'): (15, {
                word_to_index('خریدی'): (11, {}),
            }),
        })
    }),
}
print('Q[من]', q_table[word_to_index('من')][0])
print('Q[من, خوشحال]', q_table[word_to_index('من')]
      [1][word_to_index('خوشحال')][0])
print('Q[من, خوشحال, هستم]', q_table[word_to_index('من')][1]
      [word_to_index('خوشحال')][1][word_to_index('هستم')][0])
q_table


Q[من] 10
Q[من, خوشحال] 20
Q[من, خوشحال, هستم] 25


{389: (10,
  {2887: (20, {-1: (25, {0: (0, {})})}),
   10: (5, {787: (15, {-1: (10, {})}), 479: (15, {-1: (8, {})})})}),
 2172: (10,
  {2887: (20, {7602: (7, {0: (0, {})})}),
   27: (5, {787: (15, {-1: (11, {})})})})}

### Hyperparameters
You can change these parameters to get better results.

In [ ]:
q_table = dict()
alpha = 0.8
gamma = 0.95
state_N = 6
N = 75

### Q-Learning Utility Functions (50 Points)

In [ ]:
import bisect
import random
from random import choices

weights = [1 for i in range(10000)]
index = [i for i in range(10000)]
total_weights = sum(weights)
normalized_weights = [i / total_weights for i in weights]

def random_index():
    """
    returns a random index based on the weights

    Returns:
        index: index of the word
    """
    return choices(index ,normalized_weights)[0]


In [ ]:
import math
def q_table_max_find(q_table: dict[int, tuple[int, dict]], state: list[int]):
    """
    returns the index of the word with the maximum Q value in the given state. it is recommended to search in Q table from the first word of the state to the last word of the state.
    if a word is not found in the Q table, you should search in the Q table of the next word of the state and so on.
    so if we don't have Q[W_1W_2...W_N], we search for Q[W_2W_3...W_N] and so on until Q[W_N]. if we don't have Q[W_N], we should return a random index.

    Args:
        q_table: Q table
        state: list of indexes of words

    Returns:
        index: index of the word with the maximum Q value in the given state. random index if the state is not in the Q table.
    """
    max_index , max_val = None ,None
    for i in range(len(state)):
      if state[i] not in q_table : continue
      val ,next = q_table[state[i]]
      counter = i + 1
      failed = False
      while counter < len(state) and not failed :
        if state[counter] not in next: failed = True
        else : val ,next = next[state[counter]]
        counter += 1
      if failed : continue
      for index in next :
        if max_val == None or max_val <= next[index][0] :
          max_index = index  
          max_val = next[index][0]
      return random_index() if max_index is None else max_index
    
def q_table_update(q_table: dict[int, tuple[int, dict]], state: list[int]):
    """
    updates the Q table based on the given state. update the Q[W_1W_2...W_N] using the following formula:
    Q(s,a) += alpha * (reward + gamma * max_a' Q(s',a') - Q(s,a))
    where s is the state, a is the action, a' is the next action, s' is the next state, reward is the reward of the state, alpha is the learning rate, gamma is the discount factor.
    then update the Q[W_1W_2...W_{N-1}] and so on until Q[W_1].
    
    Args:
        q_table: Q table
        state: list of indexes of words
    """
    for index in range(len(state)):
      i = len(state) - index - 1  # for L(state) - 1 -> 0 
      Q = q_table
      for j in range(i):
        if state[j] not in Q : Q[state[j]] = (0 , dict())
        val ,Q = Q[state[j]]
      if state[i] not in Q : Q[state[i]] = (0 , dict())
      R = Q[state[i]][0] + alpha * (reward(state_to_sentence(state[:i]) , index_to_word(state[i])) + gamma * max([Q[i][0] for i in Q]) - Q[state[i]][0])
      Q[state[i]] = (R , Q[state[i]][1])


### Training Loop (10 Points)
Since search space is really big, we can let our model train for an hour or two and get a good result.

In [ ]:
episodes = 12000
epsilon = 1
episode_N = 75
for ep in tqdm(range(episodes)):
  state = []
  for i in range(episode_N):
    if random.random() < epsilon:
      # TODO: random action
      action = random_index()
      state.append(action)
    else:
      # TODO: greedy action with Q table max find
      action = q_table_max_find(q_table , state)
      state.append(action)
    # to avoid infinite loop
    if len(state) > 1 and state[-1] == state[-2]:
      break
    q_table_update(q_table, state)
    if len(state) > state_N:
      state = state[1:]
    if state[-1] == 0:
      break
  epsilon *= 0.99975

 20%|██        | 807/4000 [03:37<14:21,  3.71it/s]


KeyboardInterrupt: ignored

### Testing (10 Points)
This will be the final output of our model. score will be based on how well the output fits with the corpus. Generated sentences should have some meaning in the neighborhood of each word.

In [ ]:
def get_result(state, steps=75):
    for i in range(steps):
        state.append(q_table_max_find(q_table, state))
        if state[-1] == 0:
            break
        if len(state) > state_N:
            state = state[1:]
        yield state[-1]

state = sentence_to_state('ما')
print('ما', end=' ')
for s in get_result(state):
    print(words[s], end=' ')
print()
state = sentence_to_state('یک')
print('یک', end=' ')
for s in get_result(state):
    print(words[s], end=' ')
print()
state = sentence_to_state('ایران')
print('ایران', end=' ')
for s in get_result(state):
    print(words[s], end=' ')
print()